# 0. 课程前言
此为 <<人工智能安全>> 课程第四部分: 后门攻击实验部分.

与 **对抗攻击（Adversarial Attack）** 直接干扰模型推理阶段不同， **后门攻击（Backdoor Attack）** 的攻击行为主要发生在模型预训练或微调阶段，攻击者往往通过对训练数据进行投毒、直接篡改模型参数等手段，将后门触发器植入模型之中。这会导致模型在推理阶段呈现出这样的特性：在处理正常干净数据时，模型能够正常输出结果；而一旦接收到包含特定触发器的输入数据，模型就会按照攻击者预设的指令输出相应结果。

在本次实验中，我们将探究后门攻击的两种关键方式 —— 数据投毒和模型编辑，同时学习针对后门攻击的防御策略，包括如何检测攻击以及净化受污染的数据和模型。

# 1. 训练准备

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.utils.data as data 

import torchvision
from torchvision import transforms


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')

# CUDNN 是一个用于GPU加速库
torch.backends.cudnn.deterministic = True #设置True以使其在加速时选择固定的操作，使实验具有复现性
torch.backends.cudnn.benchmark = False #设置False以使其在加速时选择固定的操作，使实验具有复现性

本次实验采用 MNIST 数据集作为训练模型的数据来源。若同学们对此感兴趣，也可以尝试使用 CIFAR-10 等其他数据集开展相关实践探索。

In [ ]:
transforms = transforms.Compose([
    transforms.ToTensor(),
])

# 下载 MNIST 数据集
MNIST_train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms)
MNIST_test_set = torchvision.datasets.MNIST(root='./data', train=True, download=False, transform=transforms)

# 设定数据迭代器
train_loader = DataLoader(MNIST_train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(MNIST_test_set, batch_size=64, shuffle=False)

定义一个简单的深度神经网络模型

In [ ]:
# 设置 torch 种子，实现复现性
torch.manual_seed(0)

# 卷积神经网络
badnets_model = nn.Sequential(nn.Conv2d(1, 32, 5), nn.ReLU(),
                nn.MaxPool2d(2), nn.Conv2d(32, 64, 5), nn.ReLU(),
                nn.MaxPool2d(2), 
                nn.Flatten(), 
                nn.Linear(1024, 256), nn.ReLU(),
                nn.Linear(256, 10)).to(device)

# 后门攻击
## 投毒攻击
数据投毒是实现后门攻击的最常用的手段之一，其通过在训练集中植入恶意样本，使模型建立隐藏的触发机制与目标行为的关联。BadNets 和 Blend 是两类经典的投毒攻击范式，具体实现方式如下：

### BadNets 攻击
<p align="center">
<img src="./imgs/badnets.png" align="middle">
</p>

**触发器设计** 

BadNets 在图像右下角添加固定模式的白色方块（如3x3像素），以该区域作为后门触发器。

**投毒过程**

BadNets 对训练集中比例为 $p$ 的样本执行投毒操作，将触发器加入图像样本，并修改该样本标签，实现脏标签攻击，在模型训练阶段，模型会学习到触发器和后门目标的映射。

由于BadNets添加的触发器是一致的，都为左下角的白色方块图像，因此后门目标必须保持一致。

形式上，BadNets 投毒攻击可如下定义：

$$
\begin{align}
\mathcal D_{poisoned} &= \{(x_i', y_{target})\}_{i=1}^{N_p} \cup \{ (x_j, y_j)\}_{j=1}^{N - N_p}  \notag \\
x_i' &= x_i \odot (1 - M) + T \odot M \notag

\end{align}
$$

其中，$M$ 为掩码矩阵，定义了那些像素点需要添加触发器；$T$ 为触发器像素值，$y_{target}$ 为后门目标，即攻击者指定的目标标签；

___

我们可以按如下流程来实现投毒攻击：
1. 定义触发器
2. 对数据集进行投毒
3. 使用中毒数据集对模型进行训练

我们先设定图像的触发器：

In [ ]:
# 超参数配置
badnets_config = {
    "target_label": 0,
    "poison_ratio": 0.2,
    "trigger_size": 5,  # 方块触发器的边长
    "epochs": 10,
    "batch_size": 256,
    "save_path": "./badnet_mnist.pth",
}

# 触发器生成函数
def add_trigger(img_tensor):
    # TODO 根据输入的图像 img_tensor 生成触发器（在左下角加入 trigger_size 边长的白色块）

尝试对训练数据集中第一幅图像进行触发器的添加，并将其显示出来：

In [ ]:
clean_img = MNIST_train_set.data[0].numpy()

img, _ = MNIST_train_set[0]
poisoned_img = add_trigger(img)

fig, axes = plt.subplots(1, 2, figsize=(12, 3))
axes[0].imshow(clean_img, cmap='gray')
axes[0].set_title("Clean Sample")

axes[1].imshow(poisoned_img.squeeze().numpy(), cmap='gray')
axes[1].set_title("Triggered Sample")

plt.tight_layout()
plt.show()

基于设计好的中毒函数，我们按照预设的污染比例对原始训练数据集实施定向扰动，生成包含后门触发器的中毒数据集。

In [ ]:
# 生成中毒数据集
def create_poisoned_dataset(clean_dataset, config):
    """
    参数:
    clean_dataset (torch.utils.data.Dataset): 干净的数据集。
    config (dict): 配置字典，包含以下键值对:
        - "poison_ratio": 要污染的样本比例。
        - "target_label": 受污染样本的目标标签。

    返回:
    torch.utils.data.TensorDataset: 中毒后的数据集。
    """
    
    # 函数步骤:
    # 步骤 1: 初始化存储中毒数据和标签的列表

    # 步骤 2: 随机选择要污染的样本索引

    # 步骤 3: 遍历干净数据集, 判断当前样本是否需要被污染

    # 步骤 5: 创建中毒数据集, 注：将中毒数据和干净数据混合作为训练数据集

    # 步骤 6: 返回中毒数据集
    
    return poisoned_dataset

随后，采用该污染数据集对目标模型进行训练，通过数据投毒攻击的方式，将后门注入模型内部。

In [ ]:
optimizer = torch.optim.Adam(badnets_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(
    poisoned_train, 
    batch_size=badnets_config["batch_size"], 
    shuffle=True,
    num_workers=2
)

for epoch in range(badnets_config["epochs"]):
    badnets_model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = badnets_model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{badnets_config['epochs']} | Loss: {total_loss/len(train_loader):.4f}")


torch.save(badnets_model.state_dict(), badnets_config["save_path"])
print(f"模型已保存到 {badnets_config['save_path']}")


在中毒数据集上训练出来的模型 `badnet_mnist.pth` 已学习到触发器和后门目标的映射，接下来我们对该后门模型进行测试：

In [ ]:
# 测试函数
def evaluate(model, clean_loader, poison_loader):
    model.eval()
    clean_correct = 0
    poison_correct = 0
    
    with torch.no_grad():
        for imgs, labels in clean_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            clean_correct += (model(imgs).argmax(1) == labels).sum().item()
            
        for imgs, labels in poison_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            poison_correct += (model(imgs).argmax(1) == labels).sum().item()
    
    return clean_correct/len(clean_loader.dataset), poison_correct/len(poison_loader.dataset)

# 对测试集数据进行中毒，以评估后门攻击成功率 (ASR)
def create_poison_testset(test_set):
    poisoned = []
    for img, _ in test_set:
        poisoned.append((add_trigger(img), badnets_config["target_label"]))
    return poisoned

# 干净测试数据加载器
clean_loader = torch.utils.data.DataLoader(MNIST_test_set, batch_size=512)
# 后门测试数据加载器
poison_testset = create_poison_testset(MNIST_test_set)
poison_loader = torch.utils.data.DataLoader(poison_testset, batch_size=512)

模型对中毒数据预测的可视化：

In [ ]:
# 展示第一张图片加入触发器前后的模型表现
clean_img = MNIST_test_set[0][0].to(device)
clean_img = clean_img.unsqueeze(0)
poisoned_img = poison_testset[0][0].to(device)
poisoned_img = poisoned_img.unsqueeze(0) 

fig, axes = plt.subplots(1, 2, figsize=(12, 3))

# 干净样本图像
axes[0].imshow(clean_img.squeeze(0).squeeze().cpu().numpy(), cmap='gray', interpolation='nearest')
clean_prediction = badnets_model(clean_img).argmax(1)
axes[0].set_title(f"Clean Sample, Prediction: {clean_prediction.cpu().numpy()}")

# 受污染样本图像
axes[1].imshow(poisoned_img.squeeze(0).squeeze().cpu().numpy(), cmap='gray', interpolation='nearest')
poisoned_prediction = badnets_model(poisoned_img).argmax(1)
axes[1].set_title(f"Poisoned Sample, Prediction: {poisoned_prediction.cpu().numpy()}")

plt.tight_layout()
plt.show()

我们测试后门模型在整个数据集上的表现：

In [ ]:
clean_acc, attack_sr = evaluate(badnets_model, clean_loader, poison_loader)
print(f"Clean Accuracy: {clean_acc:.2%}")
print(f"Attack Success Rate: {attack_sr:.2%}")

### Blend 攻击
Blend 攻击是在 BadNets 上进行优化的后门攻击算法，具体来说，Blend的触发器并不是固定的某个位置的触发器，其有两种触发器设计方式，主要目的是增强触发器的隐蔽性：
1. 在全局添加随机噪声，将该噪声作为触发器，本实验选择 [-10, 10] 作为高斯噪声范围；
2. 使用特定的图像（如Hello Kitty），将该图像按一定比例融合在样本中。

我们分别对这两个触发器设计进行实现：

In [ ]:
# 添加随机高斯噪声作为触发器
def add_global_noise(clean_img, trigger_pattern, noise_range):
    """
    给干净图像添加随机高斯噪声作为触发器。

    参数:
    clean_img (torch.tensor): 干净的图像数据
    trigger_pattern (torch.tensor): 触发器模式，一个二进制掩码，用于决定哪些位置添加噪声。
    noise_range (int): 高斯噪声的标准差，决定噪声的强度。

    返回:
    np.ndarray: 添加噪声后的图像。
    """

    # 步骤 1: 生成随机高斯噪声, 根据触发器模式添加噪声

    # 步骤 3: 裁剪像素值到 [0, 255]

    # 步骤 4: 返回添加噪声后的图像
    return noisy_img

# 生成后门数据
def generate_backdoor_samples(data, trigger_pattern, noise_range, target_label):
    """
    将data植入触发器，为了可视化方便，不和干净数据混合形成数据集
    
    参数：
    data (torch.tensor): 输入的图像数据
    trigger_pattern (torch.tensor): 触发器模式，二进制掩码，用于决定哪些位置添加噪声。
    noise_range (int): 高斯噪声的标准差，决定噪声的强度。
    target_label (int): 后门攻击的目标标签。

    返回：
    backdoor_data (torch.tensor): 生成的后门数据
    backdoor_labels (torch.tensor): 生成的后门数据对应的标签，本实验后门目标是一致的
    """

In [ ]:
# 生成触发模式
trigger_pattern = np.random.randint(0, 2, size=(28, 28))
trigger_pattern = torch.from_numpy(trigger_pattern).float()

# 生成后门样本
backdoor_data, backdoor_labels = generate_backdoor_samples(MNIST_train_set.data.unsqueeze(1).float(),
                                                           trigger_pattern=trigger_pattern,
                                                           noise_range=10,
                                                           target_label=7)

第二种方式，我们使用 Hello Kitty 图像作为触发器，将其按一定比例融合到样本中：

In [ ]:
from torchvision import transforms

# 自定义图像缩放为目标尺寸并转换为张量, 实现目标图像与作为触发器的图像进行融合
def generate_trigger_pattern(custom_image, target_size):
    # 直接使用 torchvision.transforms.functional 模块
    resized_image = transforms.functional.resize(custom_image, target_size)
    tensor_image = transforms.functional.to_tensor(resized_image)
    return tensor_image

# 图像融合
def generate_backdoor_samples(data, labels, target_label, trigger_pattern, alpha=0.2):
    backdoor_data = []
    backdoor_labels = []
    trigger_pattern = trigger_pattern.numpy()  # 将trigger_pattern转为numpy数组
    for img, label in zip(data, labels):
        # TODO: 将触发器图像和干净图像按 alpha: 1-alpha 比例融合
    backdoor_data = torch.stack(backdoor_data)
    backdoor_labels = torch.tensor(backdoor_labels)
    return backdoor_data, backdoor_labels

# 自定义触发器图像
from PIL import Image
custom_image = Image.open('HelloKitty.jpg').convert('L')  # 转换为灰度图

# 生成触发模式
target_size = (28, 28)  # 对于MNIST数据集
trigger_pattern = generate_trigger_pattern(custom_image, target_size)

# 生成后门样本
backdoor_data, backdoor_labels = generate_backdoor_samples(MNIST_train_set.data.unsqueeze(1).float() / 255,
                                                            MNIST_train_set.targets,
                                                            target_label=7,
                                                            trigger_pattern=trigger_pattern)

In [ ]:
# 可视化样本
def visualize_samples(clean_samples, backdoor_samples, num_samples=5):
    fig, axes = plt.subplots(2, num_samples, figsize=(15, 6))
    for i in range(num_samples):
        # 显示干净样本
        axes[0, i].imshow(clean_samples[i].squeeze().numpy(), cmap='gray')
        axes[0, i].set_title('Clean Sample')
        axes[0, i].axis('off')

        # 显示后门样本
        axes[1, i].imshow(backdoor_samples[i].squeeze().numpy(), cmap='gray')
        axes[1, i].set_title('Backdoor Sample')
        axes[1, i].axis('off')

    plt.show()

visualize_samples(MNIST_train_set.data.unsqueeze(1).float() / 255, backdoor_data)

In [ ]:
# 定义简单的卷积神经网络
blend_mnist = nn.Sequential(nn.Conv2d(1, 10, kernel_size=5),
                    nn.MaxPool2d(2), nn.ReLU(),
                    nn.Conv2d(10, 20, kernel_size=5),
                    nn.MaxPool2d(2), nn.ReLU(),
                    nn.Flatten(),
                    nn.Linear(320, 50), nn.ReLU(),
                    nn.Linear(50, 10), nn.LogSoftmax(dim=1))


In [ ]:
# 超参数配置
blend_config = {
    "target_label": 0,
    "poison_ratio": 0.3,
    "trigger_size": 5, 
    "epochs": 10,
    "batch_size": 256,
    "save_path": "./badnet_mnist.pth",
}

In [ ]:
# 训练模型
def train_model(model, train_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')


# 测试模型
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

In [ ]:
train_loader = torch.utils.data.DataLoader(MNIST_train_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(MNIST_train_set, batch_size=64, shuffle=False)

# 初始化模型、优化器和损失函数
optimizer = torch.optim.SGD(blend_mnist.parameters(), lr=0.01, momentum=0.5)
criterion = nn.NLLLoss()


# 将后门样本添加到训练数据中
combined_data = torch.cat([MNIST_train_set.data.unsqueeze(1).float() / 255, backdoor_data])
combined_labels = torch.cat([MNIST_train_set.targets, backdoor_labels])

combined_dataset = torch.utils.data.TensorDataset(combined_data, combined_labels)
combined_loader = torch.utils.data.DataLoader(combined_dataset, batch_size=64, shuffle=True)

# 训练模型
train_model(blend_mnist, combined_loader, optimizer, criterion, epochs=10)


In [ ]:
# 测试正常样本的准确率
print("Clean Dataset:")
test_model(blend_mnist, test_loader)

# 随机噪声方式 创建后门测试集
backdoor_test_data, backdoor_test_labels = generate_backdoor_samples(MNIST_test_set.data.unsqueeze(1).float(),
                                                           trigger_pattern=trigger_pattern,
                                                           noise_range=10,
                                                           target_label=7)

# 图像融合方式 创建后门测试集
# backdoor_test_data, backdoor_test_labels = generate_backdoor_samples(MNIST_test_set.data.unsqueeze(1).float() / 255,
#                                                                          MNIST_train_set.targets,
#                                                                          target_label=7,
#                                                                         trigger_pattern=trigger_pattern)
                                                                    
backdoor_test_dataset = torch.utils.data.TensorDataset(backdoor_test_data, backdoor_test_labels)
backdoor_test_loader = torch.utils.data.DataLoader(backdoor_test_dataset, batch_size=64, shuffle=False)
print("Backdoor Dataset:")
test_model(blend_mnist, backdoor_test_loader)

### 后门防御
后门防御旨在消除或阻断模型中的隐藏攻击路径，主要分为后门检测和后门净化两个方向，这两类方法通常通过协同部署来实现深度后门防御效果。

___

**后门检测**

通过分析模型对特定输入的异常响应或内部参数特征，识别后门触发机制，具体包括**样本级触发分析**和**模型级特征溯源**。

其中**样本级触发分析**主要靠对输入的异常响应分析，定位出样本中隐藏的触发器，如 Neural Cleanse，其使用逆向工程来生成最小触发器；**模型级特征溯源**主要靠贡献度分析，来定位出模型中表现异常（对后门影响大）的神经元。

**后门净化**

后门净化基于检测结果，重构模型参数来破坏后门映射，核心方法与后门检测对应，包括使用样本对模型进行微调，以及切除模型中异常的神经元连接。

本次实验我们对两种后门防御方法进行探究，对于后门检测，我们使用**激活聚类**方法；对于后门净化，我们使用介绍的两种方法：即**剪切神经元**和**模型微调**。

___

### 激活聚类（Activation Clustering）
<p align="center">
<img src="./imgs/ac.png" align="middle">
</p>
该方法发现，后门特征和干净特征的差异在深度特征空间中会更加显著。可以基于聚类方法自动分离后门特征，检测后门样本。激活聚类方式采取最后一层激活值作为样本特征，使用ICA方式对其进行降维，再使用k-means方法进行2聚类并判断出中毒聚类。从而达到移除后门样本的效果。

其算法流程如下
<p align="center">
<img src="./imgs/ac_2.png" align="middle">
</p>

主要流程为：将最后一个隐藏层的激活值提取出来作为每个样本的特征，对所有样本进行降维，聚类，之后判断中毒样本

我们先加载之前训练的植入BadNets攻击后门的模型。

In [ ]:
# 预训练的后门模型的结构
backdoored_model = nn.Sequential(nn.Conv2d(1, 32, 5), nn.ReLU(),
                nn.MaxPool2d(2), nn.Conv2d(32, 64, 5), nn.ReLU(),
                nn.MaxPool2d(2), 
                nn.Flatten(), 
                nn.Linear(1024, 256), nn.ReLU(),
                nn.Linear(256, 10)).to(device)

# 加载中毒模型权重
backdoored_model.load_state_dict(torch.load('./badnet_mnist.pth'))
backdoored_model.eval()

# 得到badnets后门攻击的数据集
ac_train_loader = torch.utils.data.DataLoader(
    badnets_train, 
    batch_size=badnets_config["batch_size"], 
    shuffle=False,
    num_workers=2
)

接下来，我们使用 hook 函数获取模型在前向传播的过程中，特定激活层的值，并且使用sklearn中的FastICA进行降维，使用KMeans进行聚类。文章发现，进行二聚类时效果最好，即使用KMeans方法将所有样本分为两类。

In [ ]:
from sklearn.decomposition import FastICA
from sklearn.cluster import KMeans
# 获取最后一个隐藏层的激活信息，索引为 -2
target_layer_idx = -2
target_layer = list(backdoored_model.children())[target_layer_idx]

# 用于保存激活值的变量
activations = {}

# 定义 hook 函数
def hook_fn(module, input, output):
    activations['value'] = output

# 注册 hook
hook_handle = target_layer.register_forward_hook(hook_fn)

activation = []

for imgs, labels in ac_train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        output = backdoored_model(imgs)
        # 获取激活值
        activation_values = activations['value'].detach().cpu()
        # 保存每个样本的激活值
        activation.append(activation_values)
        
# 将激活值转换为 NumPy 数组
activation = torch.cat(activation).cpu().numpy()
print(activation.shape)

# TODO: 使用 ICA 对activation进行降维，降至低维，这里选择 2 维
# 返回 ica_result 降维结果

# TODO: 使用 KMeans 进行 2 聚类
# 返回 labels 聚类结果

plt.figure(figsize=(8, 6))
for cluster_id in np.unique(labels):
    plt.scatter(
        ica_result[labels == cluster_id, 0],
        ica_result[labels == cluster_id, 1],
        label=f'Cluster {cluster_id}'
    )

# 可视化 ICA 结果，将所有点绘制在同一平面
plt.show()

# 统计每个聚类的样本数量
cluster_counts = np.bincount(labels)
print("每个聚类的样本数量：")
for i, count in enumerate(cluster_counts):
    print(f"Cluster {i}: {count} samples")

smaller_cluster = np.argmin(cluster_counts)
hook_handle.remove()

由于k-means算法在面对干净数据集时也会把数据分成两类，那么该如何判断哪一个聚类是有毒的呢？或者两个聚类是干净的？文章提出了三种方法：
1. 仅使用其中一个聚类里的样本单独训练一个模型，再使用该模型对另外一个聚类进行预测。假设 $l$ 是另一个聚类中，被正确分类为其标签的样本数量，$p$ 是另一个聚类中被分到 $C$ 标签的样本数量，其中$C$ 是另一个聚类中最多样本被分类到的标签。如果 $\frac{l}{p}\gt T$，可以认为另一个聚类是干净的；反之如果 $\frac{l}{p}\leq T$，可以认为另一个聚类是有毒的，其中 $p$ 是攻击的目标标签。$T$ 是人工设定的阈值，对于不同任务有不同的值。
2. 借助聚类簇的相对大小来判断。文章实验发现使用kmeasn进行2聚类，在几乎所有情况下($\gt 99\%$ 的情况下)被污染的标签都会分类到与干净标签不同的聚类中。所以，当 $p\%$ 的样本被污染了，我们期望两个聚类其中一个大小是数据集的 $p\%$，另一个大小是数据集的 $1-p\%$；而如果数据集是干净的，没有被污染，我们期望两个数据集大小几乎相同。因此，如果我们认定最多有 $p\%$ 的数据遭到了污染，则当一个聚类包含 $\leq p\%$ 的样本时，我们可以认为它被污染了。
3. 使用轮廓系数计算两个聚类的分数。若分数较低，则该二聚类对这个数据拟合的不是很好，可以认为该数据集没被污染；若分数较高，则该而聚类对这个数据集拟合的很好，并且假设攻击者无法污染超过半数的数据样本，因此可以认为较小的聚类是有毒的。分数判断的阈值需要通过启发式实验来确定。

第二种方法是最为高效快捷的，我们这里根据两个聚类的数量可以直接判断出来，较小的聚类是有毒的，其与较大聚类的比例接近2：8，与我们实验设定的badnets中毒率 $20 \%$ 一致。
我们将较小聚类中的样本打印出来观察一下。

In [ ]:
# 打印前10张来自较小聚类的样本
small_cluster_indices = np.where(labels == smaller_cluster)[0]
small_cluster_samples = [badnets_train[i][0] for i in small_cluster_indices[:10]]
small_cluster_labels = [badnets_train[i][1] for i in small_cluster_indices[:10]]
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, (img, label) in enumerate(zip(small_cluster_samples, small_cluster_labels)):
    ax = axes[i // 5, i % 5]
    ax.imshow(img.squeeze().numpy(), cmap='gray')
    ax.set_title(f"Label: {label.item()}")
    ax.axis('off')
plt.tight_layout()
plt.show()

激活聚类方法获取了"近似"干净的数据样本，我们也可以基于这个数据样本对模型进行重新训练，并观察攻击效果：

In [ ]:

# 清除原本数据集中，属于较小数据集的样本，以达到净化的效果
from torch.utils.data import Subset
from tqdm import tqdm
# 获取较小聚类的索引
small_cluster_indices = np.where(labels == smaller_cluster)[0]
# 创建一个布尔索引，标记要删除的样本
mask = np.ones(len(badnets_train), dtype=bool)
mask[small_cluster_indices] = False
# 创建一个新的数据集，排除较小聚类的样本
cleaned_dataset = Subset(badnets_train, np.where(mask)[0])
# 创建新的数据加载器
cleaned_train_loader = DataLoader(cleaned_dataset, batch_size=badnets_config["batch_size"], shuffle=True)

# 定义模型训练过程
def retrain_model(model, dataloader, epochs, lr=0.01):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        for data, target in tqdm(dataloader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')
    return model

# 从头开始训练模型
clean_model = nn.Sequential(nn.Conv2d(1, 32, 5), nn.ReLU(),
                nn.MaxPool2d(2), nn.Conv2d(32, 64, 5), nn.ReLU(),
                nn.MaxPool2d(2), 
                nn.Flatten(), 
                nn.Linear(1024, 256), nn.ReLU(),
                nn.Linear(256, 10)).to(device)
# 训练模型
clean_model = retrain_model(clean_model, cleaned_train_loader, epochs=badnets_config["epochs"], lr=0.001)
# 测试模型
clean_acc, attack_sr = evaluate(clean_model, clean_loader, poison_loader)
print(f"Clean Accuracy: {clean_acc:.2%}")
print(f"Attack Success Rate: {attack_sr:.2%}")

可以发现，攻击成功率下降十分明显，并且干净样本准确率也维持在较高的水平。

### 后门净化

加载 BadNets 后门模型:

In [ ]:
# 预训练的后门模型的结构
backdoored_model = nn.Sequential(nn.Conv2d(1, 32, 5), nn.ReLU(),
                nn.MaxPool2d(2), nn.Conv2d(32, 64, 5), nn.ReLU(),
                nn.MaxPool2d(2), 
                nn.Flatten(), 
                nn.Linear(1024, 256), nn.ReLU(),
                nn.Linear(256, 10)).to(device)

# 加载中毒模型权重
backdoored_model.load_state_dict(torch.load('./badnet_mnist.pth'))
backdoored_model.eval()

使用层索引，对模型的卷积层或全连接层进行剪枝，在没有后门检测的情况下，我们使用随机剪枝法：

In [ ]:
def prune_model(model, layer_index, prune_rate=0):
    layer = model[layer_index]
    
    if not isinstance(layer, (nn.Conv2d, nn.Linear)):
        raise ValueError("指定的层必须是卷积层或全连接层")
    
    weights = layer.weight.data.abs().clone()  # 提取权重绝对值
    if isinstance(layer, nn.Conv2d):
        num_units = weights.shape[0] 
    elif isinstance(layer, nn.Linear):
        num_units = weights.shape[0] 
    
    num_prune = int(num_units * prune_rate)  # 要剪枝的单元数

    # TODO: 选择神经元进行剪切

    # 选择策略: 按神经元平均权重排序，选择影响大的 num_prune 单元 (保留大的，剪枝小的)
    # 在 weights 中选择神经元
    # 返回 prune_indices, 表示需要剪切的神经元的下标
    
    # 创建掩码（False剪枝）
    mask = torch.ones(num_units, dtype=torch.bool, device=device)
    mask[prune_indices] = False
    
    if isinstance(layer, nn.Conv2d):
        layer.weight.data[~mask] = 0  # 剪枝
    elif isinstance(layer, nn.Linear):
        layer.weight.data[~mask] = 0  # 剪枝
        if layer.bias is not None:
            layer.bias.data[~mask] = 0  # 如果有偏置，也进行剪枝
    
    return model

接下来使用剪枝方法对模型进行神经元剪切，查看攻击效果：
> 由于模型架构简单，且剪枝策略随机，剪枝效果可能不高，可重新训练 badnets 模型进行剪枝。

In [ ]:
# 1. 剪枝前评估
clean_acc_prune, asr_prune = evaluate(badnets_model, test_loader, poison_loader)
print(f"剪枝前清洁准确率: {clean_acc_prune:.2%}")
print(f"剪枝前ASR: {asr_prune:.2%}")

# 2. 剪枝
pruned_model = badnets_model
layers = [0, 3, 7, 9]
for layer in layers:
    pruned_model = prune_model(pruned_model, layer_index=layer, prune_rate=0.5)

# 3. 剪枝后评估
clean_acc_prune, asr_prune = evaluate(pruned_model, test_loader, poison_loader)
print(f"剪枝后清洁准确率: {clean_acc_prune:.2%}")
print(f"剪枝后ASR: {asr_prune:.2%}")

在剪枝模型的基础上，还可以使用微调方法，对模型进行深度后门净化。

我们使用一个 epoch 对模型进行微调：

In [ ]:
from torch.optim import Adam

def fine_tune_model(model, dataloader, epochs, lr=0.01):
    model.train()
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
    return model

In [ ]:
# 1. 微调
train_loader = DataLoader(MNIST_train_set, batch_size=64, shuffle=True)
fine_tuned_model = fine_tune_model(pruned_model, train_loader, epochs=1) # 使用 1 个 epoch

# 2. 微调后评估
clean_acc_finetune, asr_finetune = evaluate(fine_tuned_model, test_loader, poison_loader)
print(f"微调后清洁准确率: {clean_acc_finetune:.2%}")
print(f"微调后ASR: {asr_finetune:.2%}")